# Generate Text

### Imports

In [23]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


2022-02-16 12:25:21.491088: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-16 12:25:21.491217: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [24]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_yaml

### Load corpus

In [25]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
    corpus = corpus.replace(',', '')
    corpus = corpus.replace('.', '')
    corpus = corpus.replace('?', '')
    corpus = corpus.replace('`', '')
    corpus = corpus.replace('`', '')
    corpus = corpus.replace("'", '')
    corpus = corpus.replace('"', '')
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41191 characters


### Encoding & Decoding

In [26]:
# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

print(chars)
print("Our corpus contains {0} unique characters.".format(num_chars))
print(encoding)

['\n', ' ', '(', ')', '1', '2', ':', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Our corpus contains 33 unique characters.
{'\n': 0, ' ': 1, '(': 2, ')': 3, '1': 4, '2': 5, ':': 6, ';': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'f': 13, 'g': 14, 'h': 15, 'i': 16, 'j': 17, 'k': 18, 'l': 19, 'm': 20, 'n': 21, 'o': 22, 'p': 23, 'r': 24, 's': 25, 't': 26, 'u': 27, 'v': 28, 'w': 29, 'x': 30, 'y': 31, 'z': 32}


In [27]:
# it slices, it dices, it makes julienned datasets!
# chop up our data into X and y, slice into roughly (num_chars / skip) overlapping 'sentences'
# of length sentence_length, and encode the chars
sentence_length = 20
skip = 1
X_data = []
y_data = []

for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

In [28]:
print(len(X_data))

41171


In [29]:
X_data[1], y_data[1]

([21, 1, 32, 22, 22, 21, 1, 29, 8, 25, 1, 14, 16, 25, 26, 12, 24, 12, 21, 1],
 17)

In [30]:
X_data[2], y_data[2]

([1, 32, 22, 22, 21, 1, 29, 8, 25, 1, 14, 16, 25, 26, 12, 24, 12, 21, 1, 17],
 8)

In [31]:
num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}".format(num_sentences, sentence_length))

Sliced our corpus into 41171 sentences of length 20


In [32]:
print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=np.bool)
y = np.zeros((num_sentences, num_chars), dtype=np.bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


/var/folders/yl/xm65l11n6pb52jrsdhnd6dtw0000gn/T/ipykernel_9627/2224887287.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((num_sentences, sentence_length, num_chars), dtype=np.bool)
/var/folders/yl/xm65l11n6pb52jrsdhnd6dtw0000gn/T/ipykernel_9627/2224887287.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((num_sente

In [33]:
X[1]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, Fa

In [34]:
y[1]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [35]:
# Double check our vectorized data before we sink hours into fitting a model
print("Sanity check y. Dimension: {0} # Sentences: {1} Characters in corpus: {2}".format(y.shape, num_sentences, len(chars)))
print("Sanity check X. Dimension: {0} Sentence length: {1}".format(X.shape, sentence_length))

Sanity check y. Dimension: (41171, 33) # Sentences: 41171 Characters in corpus: 33
Sanity check X. Dimension: (41171, 20, 33) Sentence length: 20


### Model

In [36]:
# Define our model
print("Let's build model 1")
model = Sequential()
model.add(LSTM(128, input_shape=(sentence_length, num_chars)))
model.add(Dense(num_chars))
model.add(Dense(num_chars))
model.add(Dense(num_chars))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary() 

Let's build model 1
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               82944     
                                                                 
 dense_1 (Dense)             (None, 33)                4257      
                                                                 
 dense_2 (Dense)             (None, 33)                1122      
                                                                 
 dense_3 (Dense)             (None, 33)                1122      
                                                                 
 activation_2 (Activation)   (None, 33)                0         
                                                                 
Total params: 89,445
Trainable params: 89,445
Non-trainable params: 0
_________________________________________________________________


### Train

In [37]:
# Dump our model architecture to a file so we can load it elsewhere
# Find out how to load a model? ,
# return_sequences=True
architecture = model.to_json()
with open('model.json', 'w') as model_file:
    model_file.write(architecture)

# Set up checkpoints, and save trained model
file_path="weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]

# Find out how to load the trained checkpoint?
# Lets go, action time!
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks)

Epoch 1/20


2022-02-16 12:25:22.535141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-16 12:25:22.637355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-16 12:25:22.696936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


322/322 [==============================] - ETA: 0s - loss: 2.5258
Epoch 1: loss improved from inf to 2.52578, saving model to weights-01.hdf5
322/322 [==============================] - 6s 15ms/step - loss: 2.5258
Epoch 2/20
319/322 [============================>.] - ETA: 0s - loss: 2.0239
Epoch 2: loss improved from 2.52578 to 2.02400, saving model to weights-02.hdf5
322/322 [==============================] - 5s 15ms/step - loss: 2.0240
Epoch 3/20
322/322 [==============================] - ETA: 0s - loss: 1.8958
Epoch 3: loss improved from 2.02400 to 1.89580, saving model to weights-03.hdf5
322/322 [==============================] - 5s 15ms/step - loss: 1.8958
Epoch 4/20
321/322 [============================>.] - ETA: 0s - loss: 1.7996
Epoch 4: loss improved from 1.89580 to 1.79980, saving model to weights-04.hdf5
322/322 [==============================] - 5s 15ms/step - loss: 1.7998
Epoch 5/20
322/322 [==============================] - ETA: 0s - loss: 1.7134
Epoch 5: loss improved fro

In [38]:
# model.add(GRU(256),return_sequences=True)
# model.add(GRU(256))

In [39]:
corpus_length = len(corpus)
chars = sorted(list(set(corpus)))
sentence_length = 20
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

### Functions

In [40]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [41]:
def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],1.0)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=np.bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = abba_corpus[seed:seed + sentence_length]

        return pattern

### Generate Text

In [47]:
seed = make_seed('in the bard and show you on your lovelight')
print(seed)
txt =  generate(seed)
print(txt)

in the bard and show


/var/folders/yl/xm65l11n6pb52jrsdhnd6dtw0000gn/T/ipykernel_9627/3962742590.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
/var/folders/yl/xm65l11n6pb52jrsdhnd6dtw0000gn/T/ipykernel_9627/3962742590.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  activations = np.zeros((1, 1, num_

t
ik ben met me me lield
dat jij hebt n beteruwngen dan eer ling zo slecht

ik lich nu af leten
ik voel ma vool de mijsschiek een knes mijn leren staan
ik voel daar kom die zet lecht
dat ik du nogt gant 1 naar zien want geroeft
want verdat ik zoen en mij
zot het wert doe kank het straks zo een lacht
gekijk maak neer ach de kroes een vriend
zo heb ik het nooit gevoeld
nee zo had jie de mijs kranter gaan
het is toen zo er nurgenter
gaat ok heb ben zie ben ik vriegd
dat zoven mij gelooven ik heb ze


# Evaluatie

Ook vandaag vond ik het heel leuk gaan. Leuk om te zien hoe je een neurale netwerk kan bouwen en gewoon een text ermee kan generaten. Echt vet om te zien. Vandaag heb ik dus ook veel geleerd. Wel vind ik het jammer dat veel code al meegegeven was.